In [31]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3 MB 5.0 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


# OLX portal web page scraper

Upgrading pandas so there is no problem with writing

## Mount drive

In [1]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Import libraries

In [2]:
# from bs4 import BeautifulSoup

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import pandas as pd
import numpy  as np
import re
import datetime
import os,glob
import json
from json.decoder import JSONDecodeError
from tqdm import tqdm
import re


## Beautify

In [30]:
start_url= 'https://www.olx.pl/nieruchomosci/zukowo/q-dzia%C5%82ka-rolna/?search%5Border%5D=filter_float_price%3Aasc&search%5Bdist%5D=50&view=list' 

In [32]:
# makes the source tree format like

def beautify(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)

    return BeautifulSoup(page, "html.parser")

In [33]:
plots = beautify(start_url)

## Scrap table and put to lists

In [34]:
table = plots.find('table', {'class': "fixed offers breakword redesigned"})
for row in table.findAll("href"):
  print(row.get)

In [35]:
# giving all th erow also empty no text

table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [d.text.strip() for d in td]
    res.append(row)

lets remove duplicates

In [36]:
result = [] 
for nr, link in enumerate(table.find_all('a')):
  if link.get('href').__contains__('/oferta/'): # specific text which needs to be in the link
    if link.get('href') not in result:  # avoiding duplicates
      result.append(link.get('href'))
result

['https://www.olx.pl/d/oferta/dzialka-rolna-kaszuby-CID3-IDKR9oX.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-rolna-ulkowy-CID3-IDKL0l5.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-budowlana-wicko-3089m2-na-wzgorzu-blisko-leba-okazja-CID3-IDBMymF.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-budowlana-dzialki-budowlane-dzialka-rolna-CID3-IDJQ04F.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-rolna-maszewko-gm-wicko-grunt-1ha-10-000m2-krus-kopaniewo-leba-CID3-IDDMk7s.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-karwienskie-blota-2-CID3-IDK55wR.html#8f26caf238;promoted',
 'https://www.olx.pl/d/oferta/dzialka-rolna-siedlisko-w-zaciszu-z-dala-od-cywilizacji-CID3-IDK32n5.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-rolna-siedlisko-w-zaciszu-CID3-IDK2Yvd.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-rolna-w-poblizu-kartuz-CID3-IDIZz39.html#8f26caf238',
 'https://www.olx.pl/d/oferta/dzialka-rolna-2-0300-ha-CID3-IDKXhww.h

old version

In [37]:
# result = [] 
# for nr, link in enumerate(table.find_all('a')):
#   if link.get('href') not in result:
#     result.append(link.get('href'))

   
# # removing not necessary rows make it better ==with contain
# del result[2]
# del result[0]   

In [38]:
# and funcion doing removing extra spaces and new lines

def remove_extra(temp):
  temp = re.sub(r'\n+', ' ', temp) #removing extra new lines
  temp = re.sub(r' +', ' ', temp)  #removing extra spaces
  return temp

In [39]:
# retrieving info with regex to the tables
l_cena=[]
l_date_=[]
l_descr=[]

for nr, i in enumerate(res):
  if len(i)>3:
    cena= re.findall(r".+zł", i[0])
    date_= re.findall(r"\d+\s{2}\w{3}|.*\d+:\d+", i[0])
    descr= i=remove_extra(i[0])

    l_cena.append(cena)
    l_date_.append(date_)
    l_descr.append(descr)

    # print( cena[0].ljust(15), date_[0].ljust(15), descr)




## Print results from lists

In [40]:
for nr, i in enumerate(result):
  print(str(nr).ljust(2),
        result[nr].ljust(130),
        l_cena[nr],
        l_date_[nr],
        l_descr[nr])

0  https://www.olx.pl/d/oferta/dzialka-rolna-kaszuby-CID3-IDKR9oX.html#8f26caf238                                                     ['30 000 zł'] ['5  sie'] Dzialka rolna Kaszuby Działki » Sprzedaż 30 000 zł Borucino 5 sie
1  https://www.olx.pl/d/oferta/dzialka-rolna-ulkowy-CID3-IDKL0l5.html#8f26caf238                                                      ['35 000 zł'] ['12  sie'] Działka rolna Ulkowy Działki » Sprzedaż 35 000 zł Do negocjacji Ulkowy 12 sie
2  https://www.olx.pl/d/oferta/dzialka-budowlana-wicko-3089m2-na-wzgorzu-blisko-leba-okazja-CID3-IDBMymF.html#8f26caf238              ['49 999 zł'] ['6  sie'] Działka budowlana Wicko 3089m2 na wzgórzu (blisko Łeba) OKAZJA Działki » Sprzedaż 49 999 zł Do negocjacji Lębork 6 sie
3  https://www.olx.pl/d/oferta/dzialka-budowlana-dzialki-budowlane-dzialka-rolna-CID3-IDJQ04F.html#8f26caf238                         ['50 000 zł'] ['11  sie'] Działka budowlana działki budowlane działka rolna Działki » Sprzedaż 50 000 zł Mściszewice 11 sie
4

# Saving results to file

In [14]:
plots_olx_pd = pd.DataFrame({'web_link' : result,
                               'price' : l_cena,
                               'time' : l_date_,
                               'Description': l_descr})

In [ ]:
# plots_olx_pd

In [24]:
# generating file name with timestamp info for later comparison
def gen_h5_file_with_timestamp(name='name'):
    timestamp = datetime.datetime.now()
    ts1 = timestamp.strftime("%A%d%B%Y%m")
    ts2 = timestamp.strftime("%H")
    ts3 = timestamp.strftime("%M")
    ts4 = timestamp.strftime("%S")
    file_name= name + ts1 +"-"+ ts2+"-"+ ts3+"-"+ ts4+".h5"
    return file_name


In [25]:
file_name= gen_h5_file_with_timestamp(name='agro_plots_olx_')
file_name

'agro_plots_olx_Wednesday18August202108-10-26-45.h5'

In [26]:
plots_olx_pd.to_hdf('/content/drive/MyDrive/olx-scrap/scraped/'+file_name, key='df', mode='w') 

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['web_link', 'price', 'time', 'Description'], dtype='object')]

  encoding=encoding,


# Checking changes since last time

In [4]:
# preparing list of existing(previously created by scrapping) pandas linkedin .h5 files
search_dir = "/content/drive/MyDrive/olx-scrap/scraped/" #directory where from where h5 is read s
def list_per_time(search_dir=search_dir):
    files = [x for x in (filter(os.path.isfile, glob.glob(search_dir + "agro*.h5"))) ] 
    files_sorted= sorted(files, key=os.path.getmtime)
    print(files_sorted[:10])
    return files_sorted

In [5]:
# loading last( creation time) two dfs for comparison
def read_two_last_dfs():
    files_sorted=list_per_time()
    f1=files_sorted[-2:-1]
    f2=files_sorted[-1:]
    df1=pd.read_hdf(f1[0], 'df')
    df2=pd.read_hdf(f2[0], 'df')
    return df1, df2

In [6]:
df1, df2=read_two_last_dfs()

['/content/drive/MyDrive/olx-scrap/scraped/agro_plots_olx_Wednesday18August202108-10-13-54.h5', '/content/drive/MyDrive/olx-scrap/scraped/agro_plots_olx_Wednesday18August202108-10-26-45.h5']


In [8]:
print('=============links to new jobs since last checking=============\n')
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df1['web_link'])]):
    print(x, '-->',item )

=============links to new jobs since last checking=============



Checking differece to 18/08/2021

In [10]:
df_ref = pd.read_hdf('/content/drive/MyDrive/olx-scrap/scraped/agro_plots_olx_Wednesday18August202108-10-13-54.h5', 'df')
print('=============links to new jobs since 18/08/2021 =============\n')
for x,item in enumerate(df2.web_link.values[~df2['web_link'].isin(df_ref['web_link'])]):
    print(x, '-->',item )

=============links to new jobs since 18/08/2021 =============



Just out of curiosity what will be diffrence if df1 and 2 were changed places (what dissapeared...)

In [ ]:
df_ref = pd.read_hdf('/content/drive/MyDrive/olx-scrap/scraped/agro_plots_olx_Wednesday18August202108-10-13-54.h5', 'df')
print('=============links to new jobs since 18/08/2021 =============\n')
for x,item in enumerate(df_ref.web_link.values[~df_ref['web_link'].isin(df2['web_link'])]):
    print(x, '-->',item )

build the search to show when particular thing dissapeared from the list after how many days